# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [0]:
import pyspark

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("LinearReg").getOrCreate()

In [0]:
df = sqlContext.sql('select * from cruise_ship_info_csv')

In [0]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [0]:
for i in df.columns:
  print("'"+i+"'"+",")

'Ship_name',
'Cruise_line',
'Age',
'Tonnage',
'passengers',
'length',
'cabins',
'passenger_density',
'crew',


In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_cat')

In [0]:
indexed = indexer.fit(df).transform(df)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [0]:
assembler = VectorAssembler(inputCols=['Age',
'Tonnage',
'passengers',
'length',
'cabins',
'passenger_density', 'Cruise_cat'], outputCol='features')

In [0]:
output = assembler.transform(indexed)

In [0]:
output.columns

Out[91]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_cat',
 'features']

In [0]:
final_data = output.select(['features','crew'])

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
train_data,test_data = final_data.randomSplit([0.70,0.30])

In [0]:
lr = LinearRegression(labelCol='crew' )

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_result = lr_model.evaluate(test_data)

In [0]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|-0.00184042996555...|
| 0.35768651514091765|
|-0.39381200419629714|
| 0.35777369717136587|
| -0.4388243593176764|
|  1.0165960700235352|
|  -0.226203284804952|
| 0.24645614871872468|
|-0.47823553259116913|
| -0.2413187949990263|
|  0.2231756484414582|
|-0.22209975059890574|
|-0.43557172053937165|
|-0.23941823590611122|
|-0.06425911682456764|
|  0.8242587669042951|
| -1.2128879364144076|
| -1.3750053840784862|
|  0.3612731030847982|
| 0.32985812964896377|
+--------------------+
only showing top 20 rows



In [0]:
test_result.rootMeanSquaredError

Out[99]: 0.6029469187297734

In [0]:
test_result.r2

Out[100]: 0.9726383797697676

In [0]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                41|
|   mean| 8.406585365853658|
| stddev|3.6903716681088485|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [0]:
unlabelled_data = test_data.select('features')

In [0]:
prediction = lr_model.transform(unlabelled_data)

In [0]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.001840429965554|
|[5.0,115.0,35.74,...|11.842313484859082|
|[6.0,112.0,38.0,9...|11.293812004196297|
|[6.0,113.0,37.82,...|11.642226302828634|
|[6.0,158.0,43.7,1...|14.038824359317676|
|[9.0,113.0,26.74,...|11.363403929976466|
|[10.0,68.0,10.8,7...| 6.586203284804952|
|[10.0,77.0,20.16,...| 8.753543851281275|
|[10.0,86.0,21.14,...| 9.678235532591168|
|[10.0,91.62700000...| 9.241318794999026|
|[11.0,58.6,15.66,...| 7.376824351558541|
|[11.0,86.0,21.24,...| 9.522099750598906|
|[11.0,90.09,25.01...| 8.915571720539372|
|[11.0,91.62700000...| 9.239418235906111|
|[12.0,25.0,3.88,5...|2.9342591168245677|
|[12.0,77.104,20.0...| 8.765741233095705|
|[12.0,138.0,31.14...|13.062887936414407|
|[13.0,63.0,14.4,7...| 6.685005384078486|
|[13.0,101.509,27....|11.138726896915202|
|[14.0,30.27699999...| 3.400141870351036|
+--------------------+------------